# Holiday Optimiser

Inspired by [this article](http://www.capetalk.co.za/articles/10801/how-to-take-7-days-leave-in-2016-to-be-away-from-work-for-20-days) I wondered whether you could programatically determine which days to take leave in the year, in order to maximise the length of the blocks of time you are away from the office.  The code below is in Python 3 and  the explanation assumes some level of Python and pandas experience.  

__28 September 2017:__ Notebook updated to reflect 2018 holidays.

__6 July 2018:__ Notebook updates to reflect 2019 holidays.

_This is my thought process of how I got to a result, and is more than likely not the optimal way of achieving it.  The Python might be sketchy.  It's also quite a naive implementation in its approach, but seems to work.  You have been warned :)_

To start with, import the required libraries.  `pandas` will be doing the majority of the work, we need `functools` in Python 3 to use the `reduce` function, and we'll be caching the data on disk so you need the `os` module to find our file cache.

In [1]:
import pandas as pd
from functools import reduce
import os

Now we set up some variables with the country we want the holidays for, the year we are looking at and the number of days leave we have available in the year.  We'll then use the country name and year to build a file name to cache the data in.

In [2]:
country = "South Africa"
year = 2019
days_leave = 15
holidayfile = 'holidays-{}-{}.csv'.format(country.lower().replace(' ', '_'), year)

Now we grab the holidays that we need from [Office Holidays](http://www.officeholidays.com), and because we don't want to hammer their servers, store the output to disk.  On subsequent runs of the code we will then load the cached data from disk.

The code that fetches the data is simply loading the content of the URL into a pandas DataFrame - the tables in the page are returned as a list of DataFrames - there is only one table on the page, so we just grab the first one and do some cleanup, dropping an unneeded row, renaming the columns and then storing that to disk for later.

In [4]:
'http://www.officeholidays.com/countries/{}/{}.php'.format(country.lower().replace(' ', '_'), year)

'http://www.officeholidays.com/countries/south_africa/2019.php'

In [12]:
if os.path.isfile(holidayfile):
    holidays = pd.read_csv(holidayfile)
    holidays = holidays[['date','holiday']]
else:
    dfs = pd.read_html('http://www.officeholidays.com/countries/{}/{}.php'.format(country.lower().replace(' ', '_'), year))
    holidays = dfs[0].drop(0)
    holidays = holidays.drop(holidays.columns[[0,3]], axis=1)
    holidays.columns = ['date', 'holiday']
    holidays.to_csv(holidayfile)

This is what we now have in the `holidays` DataFrame:

In [13]:
holidays.head()

,date,holiday
1,March 21,Human Rights Day
2,April 19,Good Friday
3,April 22,Family Day
4,April 27,Freedom Day
5,May 01,Labour Day


The date text has been duplicated, so we need to clean that up and convert it to a DateTime.  Our date string also doesn't explicitly include the year, so we need to include that so that we get a correct DateTime.

We're splitting the `date` string on a space, grabbing the first two elements and concatenating them back together with the year.  We then convert that to a DateTime:

In [36]:
holidays.date = holidays.date.map(lambda x: '{} {}'.format(' '.join(x.split(' ')[0:2]), year))
holidays.date = pd.to_datetime(holidays.date, format="%B %d %Y")

The `date` column should now be something we can work with:

In [37]:
holidays.head()

,date,holiday
0,2018-01-01,New Year's Day
1,2018-03-21,Human Rights Day
2,2018-03-30,Good Friday
3,2018-04-02,Family Day
4,2018-04-27,Freedom Day


In order to find the days we should take leave, we first need to put together all of the days that we already have "off" - both the holidays we already have and all of the weekends.  First, create a list of all the days in the year:

In [38]:
df_year = pd.DataFrame({'date': pd.date_range(start = '{}/01/01'.format(year), end='{}/12/31'.format(year))})

Now we can merge the holidays onto the full list of dates in the year.  Because we're using the date field as the common field, we're effectively adding the `holiday` column from the `holidays` DataFrame to the full list of dates in the `df_year` DataFrame.

In [39]:
df_year = df_year.merge(holidays, how='left', on='date')
df_year.head()

,date,holiday
0,2018-01-01,New Year's Day
1,2018-01-02,NaN
2,2018-01-03,NaN
3,2018-01-04,NaN
4,2018-01-05,NaN


As we work through this, we'll have three types of days when we are "off" - weekends, public holidays and days we take leave.  Because we will be "off" when any one of those is true, I'm storing their status as a Boolean in their own column to make it easier to work with.  At the moment, any row in our DataFrame that is not null (contains something - the name of a public holiday) is a holiday.  We simply use the `map` function on the `holiday` column to return whether the value of is null or not:

In [40]:
df_year['isHoliday'] = df_year['holiday'].map(lambda x: pd.notnull(x))
df_year.head()

,date,holiday,isHoliday
0,2018-01-01,New Year's Day,True
1,2018-01-02,NaN,False
2,2018-01-03,NaN,False
3,2018-01-04,NaN,False
4,2018-01-05,NaN,False


Now any day that is a holiday is flagged as `True` and all the others are flagged as `False`.

Because the `date` column is a DateTime, we can use `dt` to access date and time properties of the column.  `df_year.date.dt.weekday_name` returns a Series of all the weekday names corresponding to the date in the `date` column, we can then call the `map` function to test if the `weekday_name` is a `Saturday` or a `Sunday` and if so, return True:

In [41]:
df_year['isWeekend'] = df_year.date.dt.weekday_name.map(lambda x: x=='Saturday' or x=='Sunday')
df_year.head(10)

,date,holiday,isHoliday,isWeekend
0,2018-01-01,New Year's Day,True,False
1,2018-01-02,NaN,False,False
2,2018-01-03,NaN,False,False
3,2018-01-04,NaN,False,False
4,2018-01-05,NaN,False,False
5,2018-01-06,NaN,False,True
6,2018-01-07,NaN,False,True
7,2018-01-08,NaN,False,False
8,2018-01-09,NaN,False,False
9,2018-01-10,NaN,False,False


Now we calculate a new column, again a Boolean, that indicates whether we have the day "off" or not - any day that is either a weekend or a public holiday is "off":

In [42]:
df_year['isDayoff'] = df_year['isWeekend'] | df_year['isHoliday']

It would be handy to have an indicator for later on why we have the day off.  To do this we call the `apply` function on the DataFrame, calling a function that returns the name of the holiday if it is a holiday, or `Weekend` if it is the weekend.  And because we've moved the holiday name into our `reason` column, we can drop the `holiday` column.

In [43]:
def makeReason(x):
    if x.isHoliday:
        return x.holiday
    if x.isWeekend:
        return 'Weekend'
    
df_year['reason'] = df_year.apply(makeReason, axis=1)
df_year = df_year.drop(['holiday'], axis=1)
df_year.head(10)

,date,isHoliday,isWeekend,isDayoff,reason
0,2018-01-01,True,False,True,New Year's Day
1,2018-01-02,False,False,False,None
2,2018-01-03,False,False,False,None
3,2018-01-04,False,False,False,None
4,2018-01-05,False,False,False,None
5,2018-01-06,False,True,True,Weekend
6,2018-01-07,False,True,True,Weekend
7,2018-01-08,False,False,False,None
8,2018-01-09,False,False,False,None
9,2018-01-10,False,False,False,None


Now things start to get a bit more complicated.  What we want to do is work out the blocks of days where we are at work, which is easy enough - we have a column called `isDayoff`, but also order them by how many working days there are in an individual block.  The logic here is that we should take the shortest working blocks first, as these are the days that are surrounded by the most days off.  The following line labels alternating blocks of on/off with increasing values in a new column called `blocklabel`:

In [44]:
df_year['blocklabel'] = (df_year.isDayoff.diff(1) != 0).astype('int').cumsum()
df_year.head(10)

,date,isHoliday,isWeekend,isDayoff,reason,blocklabel
0,2018-01-01,True,False,True,New Year's Day,1
1,2018-01-02,False,False,False,None,2
2,2018-01-03,False,False,False,None,2
3,2018-01-04,False,False,False,None,2
4,2018-01-05,False,False,False,None,2
5,2018-01-06,False,True,True,Weekend,3
6,2018-01-07,False,True,True,Weekend,3
7,2018-01-08,False,False,False,None,4
8,2018-01-09,False,False,False,None,4
9,2018-01-10,False,False,False,None,4


So the first two days of the year (off) are given the `blocklabel` value of `1`.  We then have 4 days of work, so their value of `blocklabel` is `2`, then 2 days of weekend (`blocklabel` = `3`) and so on.  Now that we have a way of identifying groups of days on and off (rows with the same value of `blocklabel`) we can group and sort them.

Since we're only interested in the days on at this point, we can drop the days off, store the days on in a new DataFrame and then group our data.  We're interested in the start and end dates of the block, as well as how many days are in that block.  We then sort the result so that we have the shortest working blocks first:

In [45]:
df_working = df_year[df_year['isDayoff'] == False]
df_blocks = pd.DataFrame({'start' : df_working.groupby('blocklabel').date.first(), 
              'end' : df_working.groupby('blocklabel').date.last(),
              'days' : df_working.groupby('blocklabel').size()
              }).sort_values(by='days').reset_index(drop=True)

df_blocks.head(10)

,days,end,start
0,1,2018-12-31,2018-12-31
1,1,2018-12-24,2018-12-24
2,1,2018-08-10,2018-08-10
3,1,2018-04-30,2018-04-30
4,2,2018-12-28,2018-12-27
5,2,2018-03-20,2018-03-19
6,2,2018-03-23,2018-03-22
7,3,2018-08-08,2018-08-06
8,3,2018-05-04,2018-05-02
9,4,2018-12-21,2018-12-18


So each one of these rows shows a block of working days - days surrounded by either holidays or weekends, ordered from smallest to biggest (with 5, a full working week) being the maximum.

The logic here (and perhaps naive logic) is that we should use up our leave by taking the smallest working blocks first.  To do this, let's grabe just those rows from the DataFrame where the sum of the days you need to take leave (the `days` column) are less than or equal to the number of days you have available (that we stored in the `days_leave` variable):

In [46]:
df_takeleave = df_blocks[df_blocks.days.cumsum() <= days_leave]

We can test that by checking how many days in total we have selected to take:

In [47]:
df_takeleave.days.sum()

13

So we've correctly selected enough rows from the blocks of work days to equal the number of days leave we have available.  But the data we have is in the form of start/end dates for the range of days we should take leave, so we need to explode that out to include the days in-between.  First, make a copy of the original dataframe, and set the index to the same as the `start` column, and name it `date`:

In [48]:
df_leavedays = df_takeleave.copy(deep=True)
df_leavedays.index = df_leavedays['start']
df_leavedays.index.name = 'date'
df_leavedays.head()

,days,end,start
date,,,
2018-12-31,1,2018-12-31,2018-12-31
2018-12-24,1,2018-12-24,2018-12-24
2018-08-10,1,2018-08-10,2018-08-10
2018-04-30,1,2018-04-30,2018-04-30
2018-12-27,2,2018-12-28,2018-12-27


Pandas has a function called `date_range` that returns (as the name suggests) a range of dates.  The function takes named paramters `start` and `end`, which is how we've named the columns in our dataframe.  So if I simply grab  those two columns and convert them to a list of dictionaries, I'll have this structure:

```
[{'end': Timestamp('2018-12-31 00:00:00'),
  'start': Timestamp('2018-12-31 00:00:00')},
 {'end': Timestamp('2018-12-24 00:00:00'),
  'start': Timestamp('2018-12-24 00:00:00')},
 {'end': Timestamp('2018-08-10 00:00:00'),
  'start': Timestamp('2018-08-10 00:00:00')},
 {'end': Timestamp('2018-04-30 00:00:00'),
  'start': Timestamp('2018-04-30 00:00:00')},
 {'end': Timestamp('2018-12-28 00:00:00'),
  'start': Timestamp('2018-12-27 00:00:00')},
 {'end': Timestamp('2018-03-20 00:00:00'),
  'start': Timestamp('2018-03-19 00:00:00')},
 {'end': Timestamp('2018-03-23 00:00:00'),
  'start': Timestamp('2018-03-22 00:00:00')},
 {'end': Timestamp('2018-08-08 00:00:00'),
  'start': Timestamp('2018-08-06 00:00:00')}]
```

We can then iterate through each dictionary in the list, passing it to `date_range` and unpacking the dictionary into named paramters.

In [50]:
df_leavedays_index = [pd.date_range(**d) for d in df_leavedays[['start','end']].to_dict('records')]
df_leavedays_index

[DatetimeIndex(['2018-12-31'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2018-12-24'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2018-08-10'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2018-04-30'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2018-12-27', '2018-12-28'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2018-03-19', '2018-03-20'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2018-03-22', '2018-03-23'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2018-08-06', '2018-08-07', '2018-08-08'], dtype='datetime64[ns]', freq='D')]

So now each item in our index corresponds to an item in the `df_leavedays` DataFrame, but where there are multiple days, those are represented in a list - we need to flatten those out, so we keep passing the `df_leavedays_index` into a DateTimeIndex via the [reduce](https://docs.python.org/2/library/functions.html#reduce) function, leaving us with a single level index of all the dates:

In [51]:
df_leavedays_index_flat = reduce(pd.tseries.index.DatetimeIndex.append, df_leavedays_index)
df_leavedays_index_flat

DatetimeIndex(['2018-12-31', '2018-12-24', '2018-08-10', '2018-04-30',
               '2018-12-27', '2018-12-28', '2018-03-19', '2018-03-20',
               '2018-03-22', '2018-03-23', '2018-08-06', '2018-08-07',
               '2018-08-08'],
              dtype='datetime64[ns]', freq=None)

Now we re-index our original `df_leavedays` DataFrame with the new index, which will give us a new DataFrame where all of the dates we should take leave are indices with missing values for all the new rows.  Remember this is what `df_leavedays` currently contains:

In [52]:
df_leavedays

,days,end,start
date,,,
2018-12-31,1,2018-12-31,2018-12-31
2018-12-24,1,2018-12-24,2018-12-24
2018-08-10,1,2018-08-10,2018-08-10
2018-04-30,1,2018-04-30,2018-04-30
2018-12-27,2,2018-12-28,2018-12-27
2018-03-19,2,2018-03-20,2018-03-19
2018-03-22,2,2018-03-23,2018-03-22
2018-08-06,3,2018-08-08,2018-08-06


In [53]:
df_leavedays = df_leavedays.reindex(df_leavedays_index_flat)
df_leavedays

,days,end,start
2018-12-31,1.0,2018-12-31,2018-12-31
2018-12-24,1.0,2018-12-24,2018-12-24
2018-08-10,1.0,2018-08-10,2018-08-10
2018-04-30,1.0,2018-04-30,2018-04-30
2018-12-27,2.0,2018-12-28,2018-12-27
2018-12-28,NaN,NaT,NaT
2018-03-19,2.0,2018-03-20,2018-03-19
2018-03-20,NaN,NaT,NaT
2018-03-22,2.0,2018-03-23,2018-03-22
2018-03-23,NaN,NaT,NaT


We don't need the `start`, `end` or `days` columns any longer, so we can drop them, and add our Boolean column (like we did for Public Holidays and Weekends):

In [54]:
df_leavedays = df_leavedays.drop(['start','end','days'], axis=1)
df_leavedays['isLeave'] = True
df_leavedays

,isLeave
2018-12-31,True
2018-12-24,True
2018-08-10,True
2018-04-30,True
2018-12-27,True
2018-12-28,True
2018-03-19,True
2018-03-20,True
2018-03-22,True
2018-03-23,True


We now want to join our leave days into our main full year DataFrame, so we need to drop the index to be a column containing DateTime values, and call the column `date` so that we can merge on it:

In [55]:
df_leavedays = df_leavedays.reset_index()
df_leavedays.columns = ['date', 'isLeave']

Then we merge as before:

In [56]:
df_year = df_year.merge(df_leavedays, how='left', on='date')
df_year.head()

,date,isHoliday,isWeekend,isDayoff,reason,blocklabel,isLeave
0,2018-01-01,True,False,True,New Year's Day,1,NaN
1,2018-01-02,False,False,False,None,2,NaN
2,2018-01-03,False,False,False,None,2,NaN
3,2018-01-04,False,False,False,None,2,NaN
4,2018-01-05,False,False,False,None,2,NaN


Now we need to update our `reason` column again, and also update our `isDayoff` column.  We now have three columns to consider (`isLeave`, `isWeekend` and `isHoliday`), so we need to create a new function.

First, convert the contents of the `isLeave` columns to Boolean (`True`/`False`) as it currently contains `True` or `NaN` values.  Then we do a boolean or on the three columns to set the value of `isDayoff` - which should be true if any of the three columns are `True`.  Lastly we update the reason text and update the `blocklabel` column since we have now added leave days into the mix:

In [57]:
df_year['isLeave'] = df_year['isLeave'].notnull()
df_year['isDayoff'] = df_year['isWeekend'] | df_year['isHoliday'] | df_year['isLeave']
def findLeave(x):
    if x.isHoliday:
        return x.reason
    if x.isWeekend:
        return 'Weekend'
    if x.isLeave:
        return 'Leave'
df_year['reason'] = df_year.apply(findLeave, axis=1)
df_year['blocklabel'] = (df_year.isDayoff.diff(1) != 0).astype('int').cumsum()
df_year.head(10)

,date,isHoliday,isWeekend,isDayoff,reason,blocklabel,isLeave
0,2018-01-01,True,False,True,New Year's Day,1,False
1,2018-01-02,False,False,False,None,2,False
2,2018-01-03,False,False,False,None,2,False
3,2018-01-04,False,False,False,None,2,False
4,2018-01-05,False,False,False,None,2,False
5,2018-01-06,False,True,True,Weekend,3,False
6,2018-01-07,False,True,True,Weekend,3,False
7,2018-01-08,False,False,False,None,4,False
8,2018-01-09,False,False,False,None,4,False
9,2018-01-10,False,False,False,None,4,False


Now we have a full calender for the year, for each day showing whether we are working or not, and if not, whether it is because it is a weekend, public holiday or because we should take it off as leave.

Ultimately we want to see how those holidays impact on our consecutive days off, so we need to pull them out and group them into the blocks of days off that contain leave.  Let's grab the days that we have off into a new DataFrame:

In [58]:
df_daysoff = df_year[df_year['isDayoff'] == True]
df_daysoff.head()

,date,isHoliday,isWeekend,isDayoff,reason,blocklabel,isLeave
0,2018-01-01,True,False,True,New Year's Day,1,False
5,2018-01-06,False,True,True,Weekend,3,False
6,2018-01-07,False,True,True,Weekend,3,False
12,2018-01-13,False,True,True,Weekend,5,False
13,2018-01-14,False,True,True,Weekend,5,False


So now we have a list off all the days off that we have in the year, including weekends, public holidays and leave, with consecutive days off having the same value of `blocklabel`.  Since we're only interested in consecutive blocks of days off that contain days that we have taken leave, we need to filter out all blocks of days that contain only weekends or public holidays.   In pandas you can apply a filter to a grouping of rows in a DataFrame, so let's group by the `blocklabel`:

In [59]:
grouped = df_daysoff.groupby('blocklabel')

Now we can define a function to filter.  When we run the `filter` method of the grouping, it passes each grouping through to the function as a DataFrame.  Our logic is:

- if all the rows have `isWeekend` set to `True` then it's a normal weekend and we can ignore it
- if none of the rows have `isLeave` set to `True` then it's a clump of public holidays and we can ignore it

In [60]:
def filter_nonleave(x):
    if x.isWeekend.all():
        return False
    if not x.isLeave.any():
        return False
    return True
result = grouped.filter(filter_nonleave)
result

,date,isHoliday,isWeekend,isDayoff,reason,blocklabel,isLeave
75,2018-03-17,False,True,True,Weekend,23,False
76,2018-03-18,False,True,True,Weekend,23,False
77,2018-03-19,False,False,True,Leave,23,True
78,2018-03-20,False,False,True,Leave,23,True
79,2018-03-21,True,False,True,Human Rights Day,23,False
80,2018-03-22,False,False,True,Leave,23,True
81,2018-03-23,False,False,True,Leave,23,True
82,2018-03-24,False,True,True,Weekend,23,False
83,2018-03-25,False,True,True,Weekend,23,False
116,2018-04-27,True,False,True,Freedom Day,33,False


Success.  As you can see the `blocklabel` column only contains 4 values - 23, 31, 59 and 97 - these are meaningless as values, but it shows that we have 4 groupings of days off, made up of consecutive weekends, public holidays and days we take leave.  We can check the numbers by grouping on the `blocklabel` again and showing the `sum`:

In [61]:
result.groupby('blocklabel').sum()

,isHoliday,isWeekend,isDayoff,isLeave
blocklabel,,,,
23,1.0,4.0,9.0,4.0
33,2.0,2.0,5.0,1.0
61,1.0,4.0,9.0,4.0
99,2.0,4.0,10.0,4.0


So in our 4 blocks we have:

- 1 public holiday, 4 days of weekend and 4 days of leave, giving 9 days off
- 2 public holidays, 2 days of weekend and 1 day1 of leave, giving 5 days off
- 1 public holiday, 4 days of weekend and 4 days of leave, giving 9 days off
- 2 public holidays, 4 days of weekend and 4 days of leave, giving 10 days off

So for 2018, the article linked right at the top can be:

> How to take 13 days leave in 2018 to be away from work for 33 days

And finally, these are the 13 days to take leave in 2018:

In [62]:
print("\n".join(result[result.isLeave == True]['date'].dt.strftime("%A %d %B").tolist()))

Monday 19 March
Tuesday 20 March
Thursday 22 March
Friday 23 March
Monday 30 April
Monday 06 August
Tuesday 07 August
Wednesday 08 August
Friday 10 August
Monday 24 December
Thursday 27 December
Friday 28 December
Monday 31 December
